In [ ]:
import time
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO
import platform
from ultralytics.utils.downloads import download
import zipfile


VAL_DIR = Path("/yolo/datasets/coco/images/val2017")
DATA_YAML_PATH = "/yolo/datasets/coco_val_only.yaml"
OUT_DIR = Path("/yolo/runs_coco_val")
OUT_DIR.mkdir(parents=True, exist_ok=True)

MODELS_TO_TEST = ["yolo11s.pt"] #"yolo11m.pt", "yolo11l.pt"
IMGSZ = 640
CONF = 0.25
LIMIT = None                     
RUN_THROUGHPUT = True            
METRICS_TXT = OUT_DIR / "metrics_small.txt"



def count_images(folder: Path) -> int:
    return sum(1 for _ in folder.glob("*.jpg"))

def export_if_needed(model: YOLO, fmt: str, name_hint: str) -> str:
    if fmt == "coreml":
        target = OUT_DIR / f"{name_hint}.mlpackage"
    elif fmt == "onnx":
        target = OUT_DIR / f"{name_hint}.onnx"
    else:
        raise ValueError("fmt must be 'coreml' or 'onnx'")
    if target.exists():
        return str(target)
    return model.export(format=fmt, imgsz=IMGSZ, nms=True)


def eval_map_and_speed(yolo_model: YOLO, project_dir: Path, run_name: str):
    r = yolo_model.val(
        data=DATA_YAML_PATH,
        split="val",
        imgsz=IMGSZ,
        plots=True,                 
        save_json=False,           
        project=str(project_dir),
        name=run_name,
        verbose=False,
    )
    spd = getattr(r, "speed", {}) or {}
    return {
        "map50_95": getattr(r.box, "map", None),
        "map50": getattr(r.box, "map50", None),
        "speed_ms_pre": spd.get("preprocess"),
        "speed_ms_inf": spd.get("inference"),
        "speed_ms_post": spd.get("postprocess"),
    }

def predict_and_time_stream(yolo_model: YOLO, name: str, engine_tag: str, val_dir: Path):
    from time import perf_counter
    t0, n = perf_counter(), 0
    for _ in yolo_model.predict(
        source=str(val_dir),
        imgsz=IMGSZ,
        conf=CONF,
        save=False,                              
        project=str(OUT_DIR / "predictions"),    
        name=f"{name}_{engine_tag}",
        stream=True,                             
        max_det=300,
        verbose=False,
    ):
        n += 1
    dt = perf_counter() - t0
    return {"images": n, "seconds": dt, "img_per_s": (n / dt) if dt else 0.0}

def log_line(s: str, fh):
    print(s)
    fh.write(s + "\n")


def ensure_coco_val_labels_from_pack(coco_root: Path):
    labels_val = coco_root / "labels" / "val2017"
    labels_val.parent.mkdir(parents=True, exist_ok=True)
    if not any(labels_val.glob("*.txt")):
        datasets_dir = coco_root.parent
        zip_path = datasets_dir / "coco2017labels.zip"
        if not zip_path.exists():
            download("https://ultralytics.com/assets/coco2017labels.zip", dir=str(datasets_dir))
        with zipfile.ZipFile(zip_path) as zf:
            zf.extractall(datasets_dir)
    try:
        (coco_root / "labels").chmod(0o755)
    except Exception:
        pass

def run():
    assert VAL_DIR.exists(), f"{VAL_DIR} does not exist"

  
    eval_dir = VAL_DIR
    if LIMIT:
        subset_dir = OUT_DIR / f"val2017_subset_{LIMIT}"
        subset_dir.mkdir(parents=True, exist_ok=True)
        if not any(subset_dir.glob("*.jpg")):
            for i, p in enumerate(sorted(VAL_DIR.glob("*.jpg"))):
                if i >= LIMIT: break
                try:
                    (subset_dir / p.name).symlink_to(p)
                except Exception:
                    import shutil; shutil.copy2(p, subset_dir / p.name)
        eval_dir = subset_dir

    coco_root = VAL_DIR.parent.parent  
    ensure_coco_val_labels_from_pack(coco_root)

    with open(METRICS_TXT, "a", encoding="utf-8") as fh:
        log_line("=" * 80, fh)
        log_line(f"Timestamp: {datetime.now().isoformat(timespec='seconds')}", fh)
        log_line(f"Host: {platform.platform()} | Python: {platform.python_version()} | Processor: {platform.processor()}", fh)
        log_line(f"VAL_DIR: {eval_dir}", fh)
        log_line(f"DATA_YAML: {DATA_YAML_PATH}", fh)
        log_line(f"Models: {', '.join(MODELS_TO_TEST)}", fh)
        log_line("-" * 80, fh)

        total_imgs = count_images(eval_dir)
        log_line(f"Discovered images: {total_imgs}", fh)

        for model_name in MODELS_TO_TEST:
            base = Path(model_name).stem
            log_line(f"\n[{base}] --- EXPORT + EVAL ---", fh)

            try:
                torch_model = YOLO(model_name)
            except Exception as e:
                log_line(f"[{base}] ERROR: failed to load weights: {e}", fh)
                continue

            # Core ML / ANE
            try:
                mlp_path = export_if_needed(torch_model, "coreml", base)
                ane = YOLO(mlp_path, task="detect")
                m_ap = eval_map_and_speed(ane, OUT_DIR / "eval", f"{base}_coreml")
                if RUN_THROUGHPUT:
                    thr = predict_and_time_stream(ane, base, "coreml", eval_dir)
                    log_line(f"[{base}][CoreML] predict: {thr['images']} imgs, {thr['seconds']:.2f}s, {thr['img_per_s']:.2f} img/s", fh)
                log_line(f"[{base}][CoreML] mAP50-95={m_ap['map50_95']:.4f} | mAP50={m_ap['map50']:.4f} | "
                         f"speed(ms) pre/inf/post={m_ap['speed_ms_pre']}/{m_ap['speed_ms_inf']}/{m_ap['speed_ms_post']}", fh)
            except Exception as e:
                log_line(f"[{base}][CoreML] ERROR: {e}", fh)

            # ONNX (CPU on macOS)
            try:
                onnx_path = export_if_needed(torch_model, "onnx", base)
                ort = YOLO(onnx_path, task="detect")
                m_ap = eval_map_and_speed(ort, OUT_DIR / "eval", f"{base}_onnx")
                if RUN_THROUGHPUT:
                    thr = predict_and_time_stream(ort, base, "onnx", eval_dir)
                    log_line(f"[{base}][ONNX]   predict: {thr['images']} imgs, {thr['seconds']:.2f}s, {thr['img_per_s']:.2f} img/s", fh)
                log_line(f"[{base}][ONNX]   mAP50-95={m_ap['map50_95']:.4f} | mAP50={m_ap['map50']:.4f} | "
                         f"speed(ms) pre/inf/post={m_ap['speed_ms_pre']}/{m_ap['speed_ms_inf']}/{m_ap['speed_ms_post']}", fh)
            except Exception as e:
                log_line(f"[{base}][ONNX]   ERROR: {e}", fh)

        log_line("\nDone. Metrics appended to: " + str(METRICS_TXT), fh)

if __name__ == "__main__":
    run()


Timestamp: 2025-10-29T22:59:48
Host: macOS-26.0.1-arm64-arm-64bit | Python: 3.11.13 | Processor: arm
VAL_DIR: /Volumes/T7 Shield/MLCommon/yolo/datasets/coco/images/val2017
DATA_YAML: /Volumes/T7 Shield/MLCommon/yolo/datasets/coco_val_only.yaml
Models: yolo11s.pt
--------------------------------------------------------------------------------
Discovered images: 5000

[yolo11s] --- EXPORT + EVAL ---
Ultralytics 8.3.220 🚀 Python-3.11.13 torch-2.9.0 CPU (Apple M1 Pro)
YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs

PyTorch: starting from 'yolo11s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (18.4 MB)


Torch version 2.9.0 has not been tested with coremltools. You may run into unexpected errors. Torch 2.5.0 is the most recent version that has been tested.



CoreML: starting export with coremltools 8.3.0...


Tuple detected at graph output. This will be flattened in the converted model.
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 166.27 passes/s]


CoreML Pipeline: starting pipeline with coremltools 8.3.0...
CoreML Pipeline: pipeline success
CoreML: export success ✅ 9.0s, saved as 'yolo11s.mlpackage' (18.3 MB)

Export complete (9.3s)
Results saved to /Volumes/T7 Shield/MLCommon/yolo
Predict:         yolo predict task=detect model=yolo11s.mlpackage imgsz=640  
Validate:        yolo val task=detect model=yolo11s.mlpackage imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app
Ultralytics 8.3.220 🚀 Python-3.11.13 torch-2.9.0 CPU (Apple M1 Pro)
Loading yolo11s.mlpackage for CoreML inference...
Setting batch=1 input of shape (1, 3, 640, 640)
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 116.0±27.5 MB/s, size: 90.5 KB)
val: Scanning /Volumes/T7 Shield/MLCommon/yolo/datasets/coco/labels/val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 5000/5000 13.1Mit/s 0.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 